In [1]:
from enum import Enum
import json
from openai.types.chat.completion_create_params import ResponseFormat
from openai.types.chat import ChatCompletionMessageParam
from models.phone import PhoneModel
from service.phone import search, PhoneFilter
from uuid import uuid4
from models.user import UserRole
from repositories.user import create as create_user, CreateUserModel
from repositories.thread import create as create_thread, CreateThreadModel
from service.store_chatbot_v2 import gen_answer
from service.openai import  _client
import random
import math
from deepeval.test_case import LLMTestCase, ConversationalTestCase

from utils import EvaluateContext


phones = search(
    PhoneFilter()
)
phone = phones[3]

class Step(str, Enum):
    GREETING_AND_PROVIDE_NEED = "greeting and provide needs about the phone"
    SEARCH_PHONE_BASE_ON_THE_BRAND = "search phone base on the brand"
    SEARCH_PHONE_BASE_ON_THE_PRICE = "search phone base on the price"
    SELECT_ONE_PHONE_FROM_THE_LIST = "select one phone from the list"
    ASK_FOR_THE_DETAILS_OF_THE_SELECTED_PHONE = "ask for the details of the selected phone"
    PROVIDE_PHONE_NUMBER = "provide phone number"
    PROVIDE_EMAIL = "provide email"

class VietnameseUserSimulator:
    def __init__(self, phone: PhoneModel):
        user_info = self.generate_user_info()

        print(f"Generated user info: {user_info}")

        self.name = user_info["name"]
        self.age = user_info["age"]
        self.gender = user_info["gender"]
        self.phone_number = user_info["phone_number"]
        self.email = user_info["email"]

        # Calculate raw budget values
        raw_min_budget = min(phone.price * 0.9, phone.price - 1000000)
        if raw_min_budget < 0:
            raw_min_budget = 0
        raw_max_budget = max(phone.price * 1.1, 0, phone.price + 1000000)

        # Round to millions
        self.min_budget = math.floor(raw_min_budget / 1000000) * 1000000
        self.max_budget = math.ceil(raw_max_budget / 1000000) * 1000000

        print(f"Original price: {phone.price:,} VND")
        print(f"Rounded budget range: {self.min_budget:,} - {self.max_budget:,} VND")

        self.basic_phone_info = (
            phone.to_text(include_key_selling_points=True)
            + f"- Brand: {phone._get_brand_name()}"
        )
        self.full_phone_info = phone.to_text(True, True, True, True)
        self.response_format: ResponseFormat = {
            "type": "json_schema",
            "json_schema": {
                "name": "Response",
                "description": "Response from you to the latest user message.",
                "strict": True,
                "schema": {
                    "type": "object",
                    "properties": {
                        "response_message": {
                            "type": "string",
                            "description": "Response message for the latest user message. It can be a question or a statement. It should be concise and in Vietnamese.",
                        },
                        "current_step_for_step": {
                            "type": "string",
                            "enum": [step.value for step in Step],
                            "description": "Current step of the response message. It should be one of the steps in the Step enum and match the current step of the conversation.",
                        },
                    },
                    "additionalProperties": False,
                    "required": ["current_step_for_step", "response_message"],
                },
            },
        }
        self.step_history: list[Step] = []
        self.conversation_history: list[ChatCompletionMessageParam] = []
        self.user = create_user(
            CreateUserModel(
                user_name=str(uuid4()), role=UserRole.chainlit_user, gender=self.gender
            )
        )
        self.thread = create_thread(
            CreateThreadModel(id=uuid4(), user_id=self.user.id, name=self.name)
        )
        self.llm_test_cases: list[LLMTestCase] = [] 

    def generate_user_info(self) -> dict:
        """Generate diverse and random Vietnamese user information"""
        
        # Vietnamese surnames (họ)
        surnames = [
            "Nguyễn", "Trần", "Lê", "Phạm", "Hoàng", "Huỳnh", "Phan", "Vũ", "Võ", "Đặng",
            "Bùi", "Đỗ", "Hồ", "Ngô", "Dương", "Lý", "Mai", "Đinh", "Lưu", "Đào",
            "Chu", "Cao", "Thái", "Tô", "Triệu", "Hà", "Lâm", "Vương", "Trịnh", "Quách"
        ]
        
        # Vietnamese middle names (tên đệm)
        male_middle_names = [
            "Văn", "Minh", "Quang", "Đức", "Hữu", "Thành", "Tuấn", "Duy", "Thanh", "Hoàng",
            "Anh", "Công", "Đình", "Xuân", "Bảo", "Hải", "Tiến", "Nam", "Tài", "Khang"
        ]
        
        female_middle_names = [
            "Thị", "Thu", "Minh", "Thanh", "Ngọc", "Hồng", "Thúy", "Kim", "Phương", "Lan",
            "Hương", "Mai", "Linh", "Trang", "Diệu", "Khánh", "Bích", "Yến", "Như", "Ái"
        ]
        
        # Vietnamese given names (tên)
        male_given_names = [
            "Anh", "Bảo", "Cường", "Dũng", "Đức", "Giang", "Hải", "Hiếu", "Hùng", "Khôi",
            "Long", "Minh", "Nam", "Phong", "Quân", "Sơn", "Tài", "Thắng", "Tuấn", "Việt",
            "Hoàng", "Kiên", "Linh", "Nghĩa", "Phúc", "Thiên", "Tùng", "Vinh", "Xuân", "Yên"
        ]
        
        female_given_names = [
            "Anh", "Bảo", "Chi", "Dung", "Giang", "Hà", "Hạnh", "Hương", "Lan", "Linh",
            "Mai", "Ngọc", "Phương", "Quỳnh", "Thảo", "Thu", "Trang", "Tuyết", "Uyên", "Yến",
            "Diệu", "Hạ", "Khánh", "Lý", "My", "Nhi", "Oanh", "Thùy", "Vân", "Xuân"
        ]
        
        # Phone number prefixes for different carriers
        phone_prefixes = [
            "032", "033", "034", "035", "036", "037", "038", "039",  # Viettel
            "070", "079", "077", "076", "078",  # Mobifone
            "083", "084", "085", "081", "082",  # Vinaphone
            "056", "058",  # Vietnamobile
            "092", "094", "088"  # Gmobile
        ]
        
        # Email domains
        email_domains = [
            "gmail.com", "yahoo.com", "hotmail.com", "outlook.com", "yandex.com",
            "icloud.com", "protonmail.com", "tutanota.com", "zoho.com", "mail.com"
        ]
        
        # Generate gender first to determine name pattern
        gender = random.choice(["male", "female"])
        
        # Generate name based on gender
        surname = random.choice(surnames)
        if gender == "male":
            middle_name = random.choice(male_middle_names)
            given_name = random.choice(male_given_names)
        else:
            middle_name = random.choice(female_middle_names)
            given_name = random.choice(female_given_names)
        
        # Sometimes skip middle name for more diversity
        if random.random() < 0.3:  # 30% chance to skip middle name
            full_name = f"{surname} {given_name}"
        else:
            full_name = f"{surname} {middle_name} {given_name}"
        
        # Generate age with weighted distribution (more young adults)
        age_ranges = [(18, 25, 0.3), (26, 35, 0.4), (36, 45, 0.2), (46, 65, 0.1)]
        age_range = random.choices(age_ranges, weights=[w for _, _, w in age_ranges])[0]
        age = random.randint(age_range[0], age_range[1])
        
        # Generate phone number
        prefix = random.choice(phone_prefixes)
        remaining_digits = ''.join([str(random.randint(0, 9)) for _ in range(7)])
        phone_number = f"0{prefix[1:]}{remaining_digits}"
        
        # Generate email with various patterns
        email_patterns = [
            lambda: f"{given_name.lower()}.{surname.lower()}",
            lambda: f"{given_name.lower()}{surname.lower()}",
            lambda: f"{surname.lower()}.{given_name.lower()}",
            lambda: f"{given_name.lower()}{random.randint(1990, 2005)}",
            lambda: f"{surname.lower()}{given_name.lower()}{random.randint(10, 99)}",
            lambda: f"{given_name.lower()}_{surname.lower()}",
        ]
        
        email_pattern = random.choice(email_patterns)
        email_username = email_pattern()
        # Remove Vietnamese accents for email
        email_username = self._remove_accents(email_username)
        email_domain = random.choice(email_domains)
        email = f"{email_username}@{email_domain}"
        
        return {
            "name": full_name,
            "age": age,
            "gender": gender,
            "phone_number": phone_number,
            "email": email
        }

    def _remove_accents(self, text: str) -> str:
        """Remove Vietnamese accents from text for email generation"""
        accent_map = {
            'à': 'a', 'á': 'a', 'ạ': 'a', 'ả': 'a', 'ã': 'a', 'â': 'a', 'ầ': 'a', 'ấ': 'a', 'ậ': 'a', 'ẩ': 'a', 'ẫ': 'a', 'ă': 'a', 'ằ': 'a', 'ắ': 'a', 'ặ': 'a', 'ẳ': 'a', 'ẵ': 'a',
            'è': 'e', 'é': 'e', 'ẹ': 'e', 'ẻ': 'e', 'ẽ': 'e', 'ê': 'e', 'ề': 'e', 'ế': 'e', 'ệ': 'e', 'ể': 'e', 'ễ': 'e',
            'ì': 'i', 'í': 'i', 'ị': 'i', 'ỉ': 'i', 'ĩ': 'i',
            'ò': 'o', 'ó': 'o', 'ọ': 'o', 'ỏ': 'o', 'õ': 'o', 'ô': 'o', 'ồ': 'o', 'ố': 'o', 'ộ': 'o', 'ổ': 'o', 'ỗ': 'o', 'ơ': 'o', 'ờ': 'o', 'ớ': 'o', 'ợ': 'o', 'ở': 'o', 'ỡ': 'o',
            'ù': 'u', 'ú': 'u', 'ụ': 'u', 'ủ': 'u', 'ũ': 'u', 'ư': 'u', 'ừ': 'u', 'ứ': 'u', 'ự': 'u', 'ử': 'u', 'ữ': 'u',
            'ỳ': 'y', 'ý': 'y', 'ỵ': 'y', 'ỷ': 'y', 'ỹ': 'y',
            'đ': 'd',
            'À': 'A', 'Á': 'A', 'Ạ': 'A', 'Ả': 'A', 'Ã': 'A', 'Â': 'A', 'Ầ': 'A', 'Ấ': 'A', 'Ậ': 'A', 'Ẩ': 'A', 'Ẫ': 'A', 'Ă': 'A', 'Ằ': 'A', 'Ắ': 'A', 'Ặ': 'A', 'Ẳ': 'A', 'Ẵ': 'A',
            'È': 'E', 'É': 'E', 'Ẹ': 'E', 'Ẻ': 'E', 'Ẽ': 'E', 'Ê': 'E', 'Ề': 'E', 'Ế': 'E', 'Ệ': 'E', 'Ể': 'E', 'Ễ': 'E',
            'Ì': 'I', 'Í': 'I', 'Ị': 'I', 'Ỉ': 'I', 'Ĩ': 'I',
            'Ò': 'O', 'Ó': 'O', 'Ọ': 'O', 'Ỏ': 'O', 'Õ': 'O', 'Ô': 'O', 'Ồ': 'O', 'Ố': 'O', 'Ộ': 'O', 'Ổ': 'O', 'Ỗ': 'O', 'Ơ': 'O', 'Ờ': 'O', 'Ớ': 'O', 'Ợ': 'O', 'Ở': 'O', 'Ỡ': 'O',
            'Ù': 'U', 'Ú': 'U', 'Ụ': 'U', 'Ủ': 'U', 'Ũ': 'U', 'Ư': 'U', 'Ừ': 'U', 'Ứ': 'U', 'Ự': 'U', 'Ử': 'U', 'Ữ': 'U',
            'Ỳ': 'Y', 'Ý': 'Y', 'Ỵ': 'Y', 'Ỷ': 'Y', 'Ỹ': 'Y',
            'Đ': 'D'
        }
        
        result = ""
        for char in text:
            result += accent_map.get(char, char)
        return result

    def get_system_prompt(self) -> list[ChatCompletionMessageParam]:
        role = (
            "# ROLE\n"
            "You are a Vietnamese virtual user playing the role of a customer searching for a new phone. You are chatting with an online customer service agent.\n"
        )
        profile = (
            f"## PROFILE\n"
            f"- Name: {self.name}\n"
            f"- Age: {self.age}\n"
            f"- Gender: {self.gender}\n"
            f"- Phone number: {self.phone_number}\n"
            f"- Email: {self.email}\n"
            f"- Min budget: {self.min_budget}\n"
            f"- Max budget: {self.max_budget}\n"
        )

        latest_step_in_past = self.step_history[-1] if self.step_history else None

        phone_looking_for = (
            (f"## INFORMATION ABOUT PHONE LOOKING FOR\n" f"{self.basic_phone_info}\n")
            if latest_step_in_past
            in [
                Step.GREETING_AND_PROVIDE_NEED,
                Step.SEARCH_PHONE_BASE_ON_THE_BRAND,
                Step.SEARCH_PHONE_BASE_ON_THE_PRICE,
                Step.SELECT_ONE_PHONE_FROM_THE_LIST,
            ]
            else (
                f"## INFORMATION ABOUT PHONE LOOKING FOR\n" f"{self.full_phone_info}\n"
            )
        )

        step_descriptions = (
            "## STEP DESCRIPTIONS\n"
            f"1. **{Step.GREETING_AND_PROVIDE_NEED.value}**: Greet the customer support agent and provide your needs about the phone. Example: 'Mình cần tư vấn điện thoại', 'Hello', 'Mình cần mua điện thoại tầm {self.min_budget} đến {self.max_budget} VNĐ', 'Xin chào', 'Tôi cần một chiếc điện thoại mới'.\n"
            f"2. **{Step.SEARCH_PHONE_BASE_ON_THE_BRAND.value}**: Search for a phone based on the brand. Example: 'Tìm điện thoại {phone._get_brand_name()}', 'Tìm điện thoại thương hiệu {phone._get_brand_name()}', '{phone._get_brand_name()}', 'hãng {phone._get_brand_name()}'.\n"
            f"3. **{Step.SEARCH_PHONE_BASE_ON_THE_PRICE.value}**: Search for a phone based on the price. Example: 'Tìm điện thoại dưới {self.min_budget} VNĐ', 'Tìm điện thoại trên {self.max_budget} VNĐ', 'Tìm điện thoại giá {self.min_budget} đến {self.max_budget} VNĐ'.\n"
            f"4. **{Step.SELECT_ONE_PHONE_FROM_THE_LIST.value}**: Select one phone from the suggested list in past. Example: 'Chọn điện thoại {phone.name} trong danh sách', 'Chọn điện thoại {phone.name}', 'cái đầu', 'mẫu số 2'.\n"
            f"5. **{Step.ASK_FOR_THE_DETAILS_OF_THE_SELECTED_PHONE.value}**: Ask for the details of the selected phone by analyzing the information in the <INFORMATION ABOUT PHONE LOOKING FOR> section. Extract key specifications, features, and selling points from this section, and formulate natural, relevant questions about these aspects. Generate diverse questions that someone would genuinely ask when considering purchasing this specific phone model. Vary your questions between technical specifications, features, promotions, colors, accessories, user experience, and purchase conditions.\n"
            f"6. **{Step.PROVIDE_PHONE_NUMBER.value}**: Provide your phone number when you need further consultation or are ready to purchase. Example: 'Số điện thoại của mình là {self.phone_number}'.\n"
            f"7. **{Step.PROVIDE_EMAIL.value}**: Provide your email. Example: 'Email của mình là {self.email}'.\n"
        )

        if latest_step_in_past:
            count = 0
            for step in reversed(self.step_history):
                if step == latest_step_in_past:
                    count += 1
                else:
                    break

            # Add special guidance for Step 5 to help generate diverse questions
            if latest_step_in_past == Step.ASK_FOR_THE_DETAILS_OF_THE_SELECTED_PHONE:
                asked_topics = self.extract_asked_topics()
                if asked_topics:
                    suggested_topics = self.suggest_new_topics(asked_topics)
                    step_descriptions += (
                        "\n## QUESTION HISTORY AND SUGGESTIONS\n"
                        f"You have already asked about: {', '.join(asked_topics)}.\n"
                        f"Consider asking about new topics such as: {', '.join(suggested_topics)}.\n"
                    )

            step_descriptions += (
                "\n## LATEST STEP IN PAST\n"
                f"Latest step in past: {latest_step_in_past.value}\n"
                f"Stay at step {latest_step_in_past.value} for {count} turns.\n"
            )

        task = (
            "## TASK\n"
            "Generate a response message for the latest user message based on the current step of the conversation. It's like talking to a real customer service agent."
        )

        guidelines = (
            "## GUIDELINES\n"
            "1. The response message should be in Vietnamese.\n"
            "2. When starting the conversation, greet the customer support agent and provide your needs about the phone. (Step 1)\n"
            "3. If the user asks for the type of product that you are looking for, provide the type of product that you are looking for is a phone.\n"
            f"4. If the user asks for the brand of the phone that you are looking for, provide the brand of the phone that you are looking for is {phone._get_brand_name()} (Step 2).\n"
            f"5. If the user asks for the price of the phone that you are looking for, provide the price of the phone that you are looking for is between {self.min_budget} and {self.max_budget} (Step 3).\n"
            f"6. If the user provides a list of phones and has a phone that you are looking for ({phone.name}), select that phone from the list (Step 4).\n"
            "7. If the user provides the details of the selected phone and asks your contact information, ask for the details of the selected phone (Step 5).\n"
            "8. If the user provides the details of the selected phone and the latest step in past is Step 5, provide your phone number or email (Step 6 or Step 7).\n"
            "\n## NOTE:\n"
            "- Imagine you are a real customer who has just interacted with a business. Your response should sound natural and authentic.\n"
            "- You need to stay at the Step 5 minimum 3 turns and maximum 5 turns before moving to Step 6 or Step 7.\n"
            f"- If you can't find the phone ({phone.name}) in the list of phones suggested by the customer service agent, you can ask for other phones (e.g., 'Có mẫu nào khác không?'). "
            "If still unavailable, then provide your contact information (Step 6 or Step 7).\n"
        )

        return [
            {"role": "system", "content": role + '\n' + profile},
            {"role": "system", "content": phone_looking_for},
            {"role": "system", "content": step_descriptions},
            {"role": "system", "content": task},
            {"role": "system", "content": guidelines},
        ]

    def extract_asked_topics(self) -> list[str]:
        """Extract topics that the user has already asked about using OpenAI API"""
        # If no conversation yet, return empty list
        if len(self.conversation_history) < 2:
            return []

        # Create a format for the response
        topic_format: ResponseFormat = {
            "type": "json_schema",
            "json_schema": {
                "name": "AskedTopics",
                "description": "Topics that the user has already asked about in the conversation",
                "strict": True,
                "schema": {
                    "type": "object",
                    "properties": {
                        "topics": {
                            "type": "array",
                            "items": {"type": "string"},
                            "description": "List of topics that have been asked about in the conversation",
                        }
                    },
                    "additionalProperties": False,
                    "required": ["topics"],
                },
            },
        }

        # Get only the last few messages to avoid token limits
        recent_messages = self.conversation_history[-10:] if len(self.conversation_history) > 10 else self.conversation_history

        # Create the prompt for OpenAI
        messages = [
            {
                "role": "system",
                "content": "You are an assistant that analyzes conversation history to identify what topics a customer has already asked about regarding a phone. Extract key topics the customer has asked about such as battery life, camera quality, screen size, price, etc."
            },
            {
                "role": "user",
                "content": f"Here is a conversation between a customer and a phone store assistant. Identify what specific topics about the phone the customer has already asked about in these messages:\n\n" + 
                           "\n".join([f"{'Customer' if msg['role'] == 'assistant' else 'Assistant'}: {msg.get('content', '')}" for msg in recent_messages])
            }
        ]

        try:
            response = _client.chat.completions.create(
                messages=messages,
                model="gpt-4.1-mini",
                temperature=0.3,
                response_format=topic_format,
                timeout=10
            )

            result = json.loads(response.choices[0].message.content or "{}")
            topics = result.get("topics", [])
            return topics[:10]  # Limit to 10 topics
        except Exception as e:
            print(f"Error extracting topics: {e}")
            # Fallback to basic topic extraction
            return ["general phone information"]

    def suggest_new_topics(self, asked_topics: list[str]) -> list[str]:
        """Suggest topics that haven't been asked about yet using OpenAI API"""
        # Setup the response format
        topic_format: ResponseFormat = {
            "type": "json_schema",
            "json_schema": {
                "name": "SuggestedTopics",
                "description": "Topics that could be asked about the phone",
                "strict": True,
                "schema": {
                    "type": "object",
                    "properties": {
                        "suggested_topics": {
                            "type": "array",
                            "items": {"type": "string"},
                            "description": "List of suggested topics about the phone that haven't been asked yet",
                        }
                    },
                    "additionalProperties": False,
                    "required": ["suggested_topics"],
                },
            },
        }

        # Create the prompt for OpenAI
        messages = [
            {
                "role": "system",
                "content": "You are an assistant that suggests relevant topics a customer could ask about a phone. Given the phone details and topics already asked, suggest new topics that would be helpful for making a purchase decision."
            },
            {
                "role": "user",
                "content": f"Phone information:\n{self.full_phone_info}\n\nTopics already asked about:\n{', '.join(asked_topics)}\n\nSuggest 5 other relevant topics the customer could ask about this phone that haven't been covered yet."
            }
        ]

        try:
            response = _client.chat.completions.create(
                messages=messages,
                model="gpt-4.1-mini",
                temperature=0.7,
                response_format=topic_format,
                timeout=10
            )

            result = json.loads(response.choices[0].message.content or "{}")
            suggested_topics = result.get("suggested_topics", [])
            return suggested_topics[:5]  # Limit to 5 topics
        except Exception as e:
            print(f"Error suggesting topics: {e}")
            # Fallback to some generic topics
            return ["special features", "warranty policy", "accessories", "user experience", "purchase options"]

    def get_next_user_message(self) -> str:

        messages: list[ChatCompletionMessageParam] = [
            *self.get_system_prompt(),
            *self.conversation_history,
        ]
        response = _client.chat.completions.create(
            messages=messages,
            model="gpt-4.1-mini",
            temperature=0.7,
            timeout=30,
            response_format=self.response_format,
        )
        response_message = response.choices[0].message.content
        parsed_information = json.loads(response_message or "{}")
        print(f"Response: {parsed_information} in turn {len(self.conversation_history) + 1}")
        current_step_for_step = parsed_information.get("current_step_for_step")
        response_message = parsed_information.get("response_message")
        if current_step_for_step and response_message:
            self.step_history.append(Step(current_step_for_step))
            self.conversation_history.append(
                {
                    "role": "assistant",
                    "content": response_message,
                }
            )
            
            evaluate_context = EvaluateContext()
            assistant_response = gen_answer(user_id=self.user.id, thread_id=self.thread.id, history=self.get_reversed_role_in_conversation_history(), evaluate_context=evaluate_context)

            self.conversation_history.append(
                {
                    "role": "user",
                    "content": assistant_response,
                }
            )
            self.llm_test_cases.append(
                LLMTestCase(
                    input=response_message,
                    actual_output= assistant_response,
                    retrieval_context= evaluate_context.knowledge,
                    additional_metadata=evaluate_context.model_dump()
                )
            )
            return response_message
        else:
            raise ValueError("Invalid response format")

    def get_reversed_role_in_conversation_history(self) -> list[ChatCompletionMessageParam]:
        reversed_history = []
        for message in self.conversation_history:
            # Only include messages that have a 'content' key
            if "content" in message and "role" in message:
                reversed_history.append({
                    "role": "user" if message["role"] == "assistant" else "assistant",
                    "content": message["content"],
                })
        return reversed_history

    def simulate_conversation(self, max_turns: int = 20):
        if not self.conversation_history:
            self.conversation_history.append(
                {
                    "role": "user",
                    "content": "Xin chào, bạn cần hỗ trợ gì ạ?",
                }
            )
        for _ in range(max_turns):
            self.get_next_user_message()
            if self.step_history[-1] == Step.PROVIDE_EMAIL or self.step_history[-1] == Step.PROVIDE_PHONE_NUMBER:
                print(f"User {self.name} has provided their contact information.")
                break
        for message in self.conversation_history:
            print(f"{message['role']}: {message.get('content')}")
        for step in self.step_history:
            print(f"Step: {step.value}")

2025-05-25 15:56:19 - Loaded .env file
2025-05-25 15:56:23 - >>> {"query": "query DefaultEntity {\n  viewer {\n    username\n    defaultEntity {\n      name\n    }\n  }\n}"}
2025-05-25 15:56:23 - >>> {"query": "query DefaultEntity {\n  viewer {\n    username\n    defaultEntity {\n      name\n    }\n  }\n}"}
2025-05-25 15:56:23 - <<< {"data":{"viewer":{"username":"phatnguyen-041203","defaultEntity":{"name":"tlcn"}}}}
2025-05-25 15:56:23 - <<< {"data":{"viewer":{"username":"phatnguyen-041203","defaultEntity":{"name":"tlcn"}}}}
weave version 0.51.48 is available!  To upgrade, please run:
 $ pip install weave --upgrade
Logged in as Weights & Biases user: phatnguyen-041203.
View Weave data at https://wandb.ai/tlcn/CHATBOT-TLCN/weave
weave version 0.51.48 is available!  To upgrade, please run:
 $ pip install weave --upgrade
Logged in as Weights & Biases user: phatnguyen-041203.
View Weave data at https://wandb.ai/tlcn/CHATBOT-TLCN/weave
2025-05-25 15:56:27 - file_cache is only supported with

🍩 https://wandb.ai/tlcn/CHATBOT-TLCN/r/call/01970828-e821-7372-af77-1b19f9ca7bf6
🍩 https://wandb.ai/tlcn/CHATBOT-TLCN/r/call/01970828-f0e5-71c3-bacd-3e056e8550cc
🍩 https://wandb.ai/tlcn/CHATBOT-TLCN/r/call/01970828-f0e5-71c3-bacd-3e056e8550cc
🍩 https://wandb.ai/tlcn/CHATBOT-TLCN/r/call/01970829-06ad-7722-90c7-bef37e70023b
🍩 https://wandb.ai/tlcn/CHATBOT-TLCN/r/call/01970829-06ad-7722-90c7-bef37e70023b
🍩 https://wandb.ai/tlcn/CHATBOT-TLCN/r/call/01970829-0c8e-7b93-9c21-93dc1dbeef27
🍩 https://wandb.ai/tlcn/CHATBOT-TLCN/r/call/01970829-0c8e-7b93-9c21-93dc1dbeef27
🍩 https://wandb.ai/tlcn/CHATBOT-TLCN/r/call/01970829-44cc-78b0-a89f-8243ebdbac61
🍩 https://wandb.ai/tlcn/CHATBOT-TLCN/r/call/01970829-44cc-78b0-a89f-8243ebdbac61
🍩 https://wandb.ai/tlcn/CHATBOT-TLCN/r/call/01970829-75db-7323-99bd-2c94d28ac9e6
🍩 https://wandb.ai/tlcn/CHATBOT-TLCN/r/call/01970829-75db-7323-99bd-2c94d28ac9e6
🍩 https://wandb.ai/tlcn/CHATBOT-TLCN/r/call/01970829-a256-7411-8367-bce7c9dc1941
🍩 https://wandb.ai/tlcn/CHAT

In [2]:
simulate_user = VietnameseUserSimulator(phone)
simulate_user.simulate_conversation()

Generated user info: {'name': 'Hồ Thành Anh', 'age': 25, 'gender': 'male', 'phone_number': '0775462858', 'email': 'anhho@yandex.com'}
Original price: 22,990,000 VND
Rounded budget range: 20,000,000 - 26,000,000 VND
🍩 https://wandb.ai/tlcn/CHATBOT-TLCN/r/call/01970828-e989-7d71-8fc5-270f6269197a
🍩 https://wandb.ai/tlcn/CHATBOT-TLCN/r/call/01970828-e989-7d71-8fc5-270f6269197a
2025-05-25 15:56:30 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-25 15:56:30 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Response: {'response_message': 'Xin chào, mình cần tư vấn mua một chiếc điện thoại mới trong tầm giá từ 20 triệu đến 26 triệu đồng.', 'current_step_for_step': 'greeting and provide needs about the phone'} in turn 2
Response: {'response_message': 'Xin chào, mình cần tư vấn mua một chiếc điện thoại mới trong tầm giá từ 20 triệu đến 26 triệu đồng.', 'current_step_for_step': 'greeting and provide needs about the phone'}

In [3]:
simulate_user.llm_test_cases

[LLMTestCase(input='Xin chào, mình cần tư vấn mua một chiếc điện thoại mới trong tầm giá từ 20 triệu đến 26 triệu đồng.', actual_output='Bạn có thương hiệu điện thoại nào trong đầu không? Ví dụ như Samsung, iPhone hay Xiaomi?', expected_output=None, context=None, retrieval_context=['- Information about your phone store:\n   - Name: FPTShop\n   - Location: https://fptshop.com.vn/cua-hang\n   - Hotline: 1800.6601\n   - Website: [FPTShop](https://fptshop.com.vn)\n   - Customer service email: cskh@fptshop.com\n\n- Current date: Sunday, May 25, 2025 (2025-05-25)'], additional_metadata={'instruction': '## INSTRUCTIONS:\n0. You must ask the user for the brand of the phone they are interested in such as Samsung, iPhone, etc.:\n', 'knowledge': ['- Information about your phone store:\n   - Name: FPTShop\n   - Location: https://fptshop.com.vn/cua-hang\n   - Hotline: 1800.6601\n   - Website: [FPTShop](https://fptshop.com.vn)\n   - Customer service email: cskh@fptshop.com\n\n- Current date: Sunday,

In [4]:
print(phone.to_text(True, True, True, True))

Phone: [iPhone 16](https://fptshop.com.vn/dien-thoai/iphone-16)
- Original price: 22990000. Sale price: 19190000
- Key selling points:  Chip Apple A18 mạnh mẽ, Thời lượng pin 22 giờ, Nút Camera  Control mới
- Promotions:
 - Chủ thẻ Nam Á Bank: Giảm ngay 1.000.000đ
 - Chủ thẻ NCB: Giảm ngay 1.000.000đ
 - Giảm 5% tối đa 150K cho đơn từ 55K Hoặc Giảm 20% tối đa 50K cho đơn từ 65K
 - Chủ thẻ HD Bank: Giảm 500,000đ đơn từ 5 triệu.
 - Giảm đến 700.000đ qua thẻ Muadee by HDBank
 - Giảm 500.000đ trả góp qua thẻ Visa
 - Giảm 5% tối đa 200.000đ Hoặc Giảm 50% tối đa 100.000đ qua Kredivo
- Variants: Trắng (color) - 128 GB (rom), Xanh Lưu Ly (color) - 128 GB (rom), Hồng (color) - 128 GB (rom), Xanh Mòng Két (color) - 128 GB (rom), Hồng (color) - 256 GB (rom), Đen (color) - 128 GB (rom), Xanh Lưu Ly (color) - 256 GB (rom), Xanh Mòng Két (color) - 256 GB (rom), Trắng (color) - 256 GB (rom), Đen (color) - 256 GB (rom), Đen (color) - 512 GB (rom), Hồng (color) - 512 GB (rom)
- Description: [Đánh dấu cu

In [5]:
from deepeval.evaluate.evaluate import evaluate
from deepeval.test_case import LLMTestCase, ConversationalTestCase
from deepeval.metrics import RoleAdherenceMetric
import deepeval.models.llms.openai_model as openai_model
from deepeval.metrics import FaithfulnessMetric

for llm_test_case in simulate_user.llm_test_cases:
    retrieval_context = "\n\n".join(llm_test_case.retrieval_context) if llm_test_case.retrieval_context else ""

    instruction = llm_test_case.additional_metadata.get("instruction", "") if llm_test_case.additional_metadata else ""

    chatbot_role = f"""
    # ROLE
    You are professional sales consultant staff for a laptop store.

    {retrieval_context}

    {instruction}
    
    ## TASK
    Your task is to assist users in selecting suitable laptops and providing guidance on purchasing procedures.
    Base on <INSTRUCTIONS> to provide the response for user.
    """

    convo_test_case = ConversationalTestCase(chatbot_role=chatbot_role, turns=[llm_test_case])
    role_adherence_metric = RoleAdherenceMetric(threshold=0.5, model="gpt-4.1-mini")
    faithfulness_metric = FaithfulnessMetric(threshold=0.5, model="gpt-4.1-mini")
    faiithfullness_test_case = LLMTestCase(
        input=llm_test_case.input,
        actual_output=llm_test_case.actual_output,
        retrieval_context=(llm_test_case.retrieval_context or []) + [instruction] if instruction else [],
    )
    role_adherence_metric.measure(convo_test_case)
    faithfulness_metric.measure(faiithfullness_test_case)
    print("RoleAdherenceMetric:", role_adherence_metric.score)
    print("FaithfulnessMetric:", faithfulness_metric.score)
    print(f"Input: {llm_test_case.input}")
    print(f"Actual output: {llm_test_case.actual_output}")
    if faithfulness_metric.score is not None and faithfulness_metric.score < 1:
        print(f"Faithfulness issue detected for input: {faithfulness_metric.reason}")
    if role_adherence_metric.score is not None and role_adherence_metric.score < 0.5:
        print(chatbot_role)

Output()

🍩 https://wandb.ai/tlcn/CHATBOT-TLCN/r/call/0197082d-37fb-7ac3-90a7-c02e4496071a

2025-05-25 16:01:11 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


🍩 https://wandb.ai/tlcn/CHATBOT-TLCN/r/call/0197082d-3c29-7003-afcd-91bbd995a127

2025-05-25 16:01:14 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

2025-05-25 16:08:35 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


🍩 https://wandb.ai/tlcn/CHATBOT-TLCN/r/call/01970834-02d9-7491-b69d-81b81af2c3f6

2025-05-25 16:08:37 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


2025-05-25 16:08:40 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


RoleAdherenceMetric: 1.0
FaithfulnessMetric: 1.0
Input: Mình muốn hỏi thêm về khả năng tản nhiệt và độ bền của iPhone 16 khi sử dụng liên tục, đặc biệt khi chơi game hoặc chạy ứng dụng nặng. Máy có bị nóng không và có thiết kế gì giúp cải thiện vấn đề này?
Actual output: iPhone 16 được thiết kế với khả năng tản nhiệt tốt, nhờ vào cấu trúc bên trong được tái thiết kế để nâng cao hiệu suất tản nhiệt. Điều này giúp máy duy trì hiệu suất ổn định khi sử dụng liên tục, đặc biệt là khi chơi game hoặc chạy các ứng dụng nặng. 

Với chất liệu nhôm đạt chuẩn hàng không vũ trụ và lớp kính Ceramic Shield, iPhone 16 không chỉ bền bỉ mà còn giúp giảm thiểu tình trạng nóng máy trong quá trình sử dụng.

Nếu bạn cần thêm thông tin chi tiết hoặc muốn đặt hàng, hãy cho mình biết số điện thoại hoặc email của bạn để mình có thể tư vấn tốt hơn nhé!
